<b>Step 1 :</b>- Importez les bibliothèques requises et lisez les jeux de données de test et d'entraînement. Ajouter les deux.

In [ ]:
# Importer les bibliothèques requises 
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Lire le jeu de données de test et d'entrainement
##############################################################
######## Changer les chemins vers les jeux de données ########
##############################################################
train=pd.read_csv('C:/Users/Analytics Vidhya/Desktop/challenge/Train.csv')
test=pd.read_csv('C:/Users/Analytics Vidhya/Desktop/challenge/Test.csv')
# Create a flag for Train and Test Data set
train['Type']='Train' 
test['Type']='Test'
# Assembler les deux
fullData = pd.concat([train,test],axis=0) 

<b>Étape 2 :</b> L'étape 2 du cadre n'est pas requise en Python. Passons à l'étape suivante.

<b>Étape 3 :</b> Affichez les noms de colonne / résumé de l'ensemble de données.

In [ ]:
# This will show all the column names
fullData.columns 

# Show first 10 records of dataframe
fullData.head(10) 

#You can look at summary of numerical fields by using describe() function
fullData.describe()

<b>Étape 4 :</b> Identifiez les a) variables ID b) les variables cibles c) les variables qualitatives d) les variables numériques e) les autres variables

In [ ]:
ID_col = ['REF_NO']
target_col = ["Account.Status"]
cat_cols = ['children','age_band','status','occupation','occupation_partner','home_status','family_income','self_employed', 'self_employed_partner','year_last_moved','TVarea','post_code','post_area','gender','region']
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col)-set(data_col))
other_col=['Type'] #Test and Train Data set identifier

<b>Step 5 :<b> Identifiez les variables avec les valeurs manquantes et créez un indicateur pour celles-ci.

In [ ]:
#Will return the feature with True or False,True means have missing value else False
fullData.isnull().any()

In [ ]:
# Combined numerical and Categorical variables
num_cat_cols = num_cols+cat_cols 

#Create a new variable for each variable having missing value with VariableName_NA 
# and flag missing value with 1 and other with 0

for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1 


<b>Step 6 :</b> Imputer les valeurs manquantes

In [ ]:
#Impute numerical missing values with mean
fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean(),inplace=True)

In [ ]:
#Impute categorical missing values with -9999
fullData[cat_cols] = fullData[cat_cols].fillna(value = -9999)

<b>Step 7 :</b> Créez des encodeurs de labels pour les variables qualitatives et divisez le jeu de données pour l'apprentissage et le teste, puis divisez le jeu de données d'entrainenement en Train et validez.

In [ ]:
#create label encoders for categorical features
for var in cat_cols:
 number = LabelEncoder()
 fullData[var] = number.fit_transform(fullData[var].astype('str'))

#Target variable is also a categorical so convert it
fullData["Account.Status"] = number.fit_transform(fullData["Account.Status"].astype('str'))

train=fullData[fullData['Type']=='Train']
test=fullData[fullData['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

<b>Step 8 :</b> Pass the imputed and dummy (missing values flags) variables into the modelling process. I am using random forest to predict the class

In [ ]:
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(other_col))

In [ ]:
x_train = Train[list(features)].values
y_train = Train["Account.Status"].values
x_validate = Validate[list(features)].values
y_validate = Validate["Account.Status"].values
x_test=test[list(features)].values

In [ ]:
random.seed(100)
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

<b>Step 9 :</b> Check performance and make predictions

In [ ]:
status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1])
roc_auc = auc(fpr, tpr)
print roc_auc

final_status = rf.predict_proba(x_test)
test["Account.Status"]=final_status[:,1]
test.to_csv('C:/Users/Analytics Vidhya/Desktop/model_output.csv',columns=['REF_NO','Account.Status'])